In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from bokeh.io import output_notebook
from importlib import reload
%matplotlib inline

import moving_average as ma
import macd
import utility

/home/ryohayama/.local/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [2]:
data = pd.read_csv('fx_data/DAT_ASCII_USDJPY_M1_2017.csv', sep=';', header=None)
data.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
data['time'] = pd.to_datetime(data['time'], infer_datetime_format=True)
data = data.set_index('time')
del data['volume']

In [3]:
data.columns

Index(['open', 'high', 'low', 'close'], dtype='object')

In [4]:
aggDay= utility.aggregate_periods(data, period=60)
print(len(aggDay))

6189


In [5]:
aggDay.iloc[:500]

open     high      low    close
time                                                   
2017-01-02 03:03:00  116.858  117.169  116.858  117.167
2017-01-02 04:04:00  117.172  117.446  117.167  117.408
2017-01-02 05:06:00  117.403  117.436  117.292  117.409
2017-01-02 06:09:00  117.410  117.434  117.316  117.320
2017-01-02 07:11:00  117.326  117.408  117.316  117.386
2017-01-02 08:12:00  117.397  117.494  117.383  117.452
2017-01-02 09:13:00  117.454  117.457  117.318  117.366
2017-01-02 10:13:00  117.364  117.395  117.349  117.370
2017-01-02 11:13:00  117.379  117.422  117.321  117.323
2017-01-02 12:16:00  117.341  117.402  117.309  117.402
2017-01-02 13:18:00  117.399  117.462  117.334  117.400
2017-01-02 14:22:00  117.406  117.498  117.397  117.458
2017-01-02 15:22:00  117.464  117.654  117.458  117.599
2017-01-02 16:23:00  117.600  117.641  117.544  117.602
2017-01-02 17:24:00  117.598  117.662  117.428  117.453
2017-01-02 18:24:00  117.448  117.448  117.201  117.333
2017-01-02 19:24:00  117.334  117.525  117.309  117.398
2017-01-02 20:24:00  117.403  117.454  117.280  117.340
2017-01-02 21:24:00  117.352  117.438  117.284  117.378
2017-01-02 22:27:00  117.383  117.413  117.355  117.376
2017-01-02 23:28:00  117.369  117.399  117.350  117.361
2017-01-03 00:28:00  117.355  117.376  117.288  117.334
2017-01-03 01:28:00  117.333  117.370  117.314  117.337
2017-01-03 02:28:00  117.333  117.795  117.322  117.765
2017-01-03 03:28:00  117.767  117.995  117.749  117.964
2017-01-03 04:28:00  117.966  118.121  117.862  118.096
2017-01-03 05:28:00  118.136  118.198  118.065  118.144
2017-01-03 06:28:00  118.141  118.348  118.126  118.326
2017-01-03 07:28:00  118.334  118.343  118.200  118.298
2017-01-03 08:28:00  118.300  118.313  118.142  118.246
...                      ...      ...      ...      ...
2017-01-29 17:53:00  115.080  115.118  114.723  114.873
2017-01-29 18:53:00  114.875  114.951  114.573  114.645
2017-01-29 19:53:00  114.644  114.753  114.581  114.678
2017-01-29 20:53:00  114.698  114.708  114.352  114.396
2017-01-29 21:53:00  114.391  114.460  114.349  114.374
2017-01-29 22:53:00  114.361  114.401  114.263  114.382
2017-01-29 23:53:00  114.369  114.538  114.369  114.485
2017-01-30 00:53:00  114.468  114.536  114.411  114.528
2017-01-30 01:53:00  114.521  114.659  114.518  114.526
2017-01-30 02:53:00  114.533  114.757  114.533  114.709
2017-01-30 03:53:00  114.715  114.935  114.693  114.831
2017-01-30 04:53:00  114.827  114.839  114.635  114.666
2017-01-30 05:53:00  114.655  114.744  114.513  114.565
2017-01-30 06:53:00  114.580  114.745  114.553  114.637
2017-01-30 07:53:00  114.663  114.778  114.608  114.611
2017-01-30 08:53:00  114.631  114.639  114.350  114.511
2017-01-30 09:53:00  114.506  114.519  114.172  114.331
2017-01-30 10:53:00  114.323  114.336  113.736  113.823
2017-01-30 11:53:00  113.769  113.926  113.658  113.783
2017-01-30 12:53:00  113.800  113.876  113.450  113.589
2017-01-30 13:53:00  113.602  113.733  113.573  113.654
2017-01-30 14:53:00  113.652  113.797  113.594  113.740
2017-01-30 15:53:00  113.728  113.743  113.533  113.728
2017-01-30 16:53:00  113.734  113.820  113.695  113.814
2017-01-30 17:55:00  113.813  113.814  113.684  113.708
2017-01-30 18:55:00  113.702  113.840  113.660  113.682
2017-01-30 19:55:00  113.665  113.804  113.581  113.729
2017-01-30 20:55:00  113.761  113.832  113.636  113.661
2017-01-30 21:55:00  113.638  113.708  113.460  113.570
2017-01-30 22:55:00  113.648  113.732  113.241  113.555

[500 rows x 4 columns]

In [12]:
reload(macd)
reload(utility)
MACD = macd.MACD(aggDay.iloc[:200])
MACD.plot()

/home/ryohayama/.local/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Loading BokehJS ...

In [7]:
MACD.getCrossSignals()

In [8]:
MACD.data

open     high      low    close  numerical_index  \
time                                                                       
2017-01-02 03:03:00  116.858  117.169  116.858  117.167                0   
2017-01-02 04:04:00  117.172  117.446  117.167  117.408                1   
2017-01-02 05:06:00  117.403  117.436  117.292  117.409                2   
2017-01-02 06:09:00  117.410  117.434  117.316  117.320                3   
2017-01-02 07:11:00  117.326  117.408  117.316  117.386                4   
2017-01-02 08:12:00  117.397  117.494  117.383  117.452                5   
2017-01-02 09:13:00  117.454  117.457  117.318  117.366                6   
2017-01-02 10:13:00  117.364  117.395  117.349  117.370                7   
2017-01-02 11:13:00  117.379  117.422  117.321  117.323                8   
2017-01-02 12:16:00  117.341  117.402  117.309  117.402                9   
2017-01-02 13:18:00  117.399  117.462  117.334  117.400               10   
2017-01-02 14:22:00  117.406  117.498  117.397  117.458               11   
2017-01-02 15:22:00  117.464  117.654  117.458  117.599               12   
2017-01-02 16:23:00  117.600  117.641  117.544  117.602               13   
2017-01-02 17:24:00  117.598  117.662  117.428  117.453               14   
2017-01-02 18:24:00  117.448  117.448  117.201  117.333               15   
2017-01-02 19:24:00  117.334  117.525  117.309  117.398               16   
2017-01-02 20:24:00  117.403  117.454  117.280  117.340               17   
2017-01-02 21:24:00  117.352  117.438  117.284  117.378               18   
2017-01-02 22:27:00  117.383  117.413  117.355  117.376               19   
2017-01-02 23:28:00  117.369  117.399  117.350  117.361               20   
2017-01-03 00:28:00  117.355  117.376  117.288  117.334               21   
2017-01-03 01:28:00  117.333  117.370  117.314  117.337               22   
2017-01-03 02:28:00  117.333  117.795  117.322  117.765               23   
2017-01-03 03:28:00  117.767  117.995  117.749  117.964               24   
2017-01-03 04:28:00  117.966  118.121  117.862  118.096               25   
2017-01-03 05:28:00  118.136  118.198  118.065  118.144               26   
2017-01-03 06:28:00  118.141  118.348  118.126  118.326               27   
2017-01-03 07:28:00  118.334  118.343  118.200  118.298               28   
2017-01-03 08:28:00  118.300  118.313  118.142  118.246               29   
...                      ...      ...      ...      ...              ...   
2017-12-28 12:29:00  112.855  112.878  112.822  112.829             6159   
2017-12-28 13:29:00  112.828  112.884  112.822  112.861             6160   
2017-12-28 14:29:00  112.866  112.887  112.840  112.865             6161   
2017-12-28 15:29:00  112.870  112.900  112.855  112.868             6162   
2017-12-28 16:32:00  112.874  112.894  112.836  112.858             6163   
2017-12-28 17:32:00  112.858  112.910  112.822  112.908             6164   
2017-12-28 18:36:00  112.856  112.913  112.841  112.878             6165   
2017-12-28 19:37:00  112.873  112.968  112.862  112.935             6166   
2017-12-28 20:37:00  112.933  112.936  112.702  112.763             6167   
2017-12-28 21:37:00  112.761  112.777  112.683  112.744             6168   
2017-12-28 22:38:00  112.741  112.789  112.687  112.770             6169   
2017-12-28 23:38:00  112.775  112.788  112.734  112.748             6170   
2017-12-29 00:38:00  112.749  112.766  112.726  112.728             6171   
2017-12-29 01:38:00  112.731  112.750  112.686  112.716             6172   
2017-12-29 02:38:00  112.709  112.746  112.635  112.711             6173   
2017-12-29 03:38:00  112.710  112.727  112.567  112.577             6174   
2017-12-29 04:38:00  112.569  112.603  112.548  112.553             6175   
2017-12-29 05:38:00  112.554  112.609  112.522  112.543             6176   
2017-12-29 06:38:00  112.538  112.543  112.500  112.510             6177   
2017-12-29 07:38:00  112.515  112.545  112.470  112.534             6178